In [3]:
config = """
Global:

Train:
  dataset:
    transforms:
      - DecodeImage: # load image
          img_mode: RGB
          channel_first: False
      - VQATokenLabelEncode: # Class handling label
          contains_re: True
          algorithm: "Bert"
          class_path: train_data/XFUND/class_list_xfun.txt
          use_textline_bbox_info: True
          order_method: "tb-yx"
      - VQATokenPad:
          max_seq_len: 512
          return_attention_mask: True
      - VQAReTokenRelation:
      - VQAReTokenChunk:
          max_seq_len: 512
      - TensorizeEntitiesRelations:
      - Resize:
          size: [224,224]
      - NormalizeImage:
          scale: 1
          mean: [ 123.675, 116.28, 103.53 ]
          std: [ 58.395, 57.12, 57.375 ]
          order: 'hwc'
      - ToCHWImage:
      - KeepKeys:
          keep_keys: [ 'input_ids', 'bbox','attention_mask', 'token_type_ids', 'entities', 'relations']
"""

import yaml

config = yaml.load(config, Loader=yaml.FullLoader)
dataset_config = config['Train']['dataset']
global_config = config['Global']

In [4]:
import json
import os
from ppocr.data.imaug import create_operators, transform
import paddle

data_dir = "train_data/XFUND/zh_train/image"
label_file = "train_data/XFUND/zh_train/train.json"
with open(label_file, 'r') as f:
    lines = f.readlines()

line = lines[1].strip()
file_name, label = line.split('\t')
img_path = os.path.join(data_dir, file_name)

data = {'img_path': img_path, 'label': label}
with open(data['img_path'], 'rb') as f:
    img = f.read()
    data['image'] = img

ops = create_operators(dataset_config['transforms'], global_config)

outs = transform(data, ops)
outs = [paddle.to_tensor(out, place='cpu') for out in outs]
batch = [paddle.unsqueeze(out, axis=0) for out in outs]

[2022-12-15 00:59:02,788] [    INFO] - We are using <class 'paddlenlp.transformers.bert.tokenizer.BertTokenizer'> to load 'bert-base-chinese'.
[2022-12-15 00:59:02,788] [    INFO] - Already cached /home/kaihong/.paddlenlp/models/bert-base-chinese/bert-base-chinese-vocab.txt
[2022-12-15 00:59:02,797] [    INFO] - tokenizer config file saved in /home/kaihong/.paddlenlp/models/bert-base-chinese/tokenizer_config.json
[2022-12-15 00:59:02,797] [    INFO] - Special tokens file saved in /home/kaihong/.paddlenlp/models/bert-base-chinese/special_tokens_map.json


In [5]:
print(f"type of outs: {type(batch)}")
print(f"len of outs: {len(batch)}")

print(f"shape of input_ids: {batch[0].shape}")
print(f"shape of bbox: {batch[1].shape}")
print(f"shape of attention_mask: {batch[2].shape}")
print(f"shape of token_type_ids: {batch[3].shape}")
print(f"shape of entities: {batch[4].shape}")
print(f"shape of relations: {batch[5].shape}")

# relations = batch[5].squeeze().numpy()
# relations
# import matplotlib.pyplot as plt
# import matplotlib.patches as patches
# plot scatter plot
# x = relations[1:, 0]
# y = relations[1:, 1]
# plt.scatter(x, y, s=1)


type of outs: <class 'list'>
len of outs: 6
shape of input_ids: [1, 512]
shape of bbox: [1, 512, 4]
shape of attention_mask: [1, 512]
shape of token_type_ids: [1, 512]
shape of entities: [1, 513, 3]
shape of relations: [1, 262145, 2]


In [1]:
from ppocr.modeling.architectures import build_model
from ppocr.modeling.backbones import build_backbone

config = {
    'model_type': 'kie',
    'Transform': None,
    'Backbone': {
        'name': 'BertForRe',
        'pretrained': True,
        'mode': 'base',
        'checkpoint': None
    }
}

model = build_model(config)


[2022-12-15 00:58:52,233] [    INFO] - Already cached /home/kaihong/.paddlenlp/models/bert-base-chinese/bert-base-chinese.pdparams
W1215 00:58:52.234423 1370870 gpu_resources.cc:61] Please NOTE: device: 0, GPU Compute Capability: 7.5, Driver API Version: 12.0, Runtime API Version: 11.6
W1215 00:58:52.236507 1370870 gpu_resources.cc:91] device: 0, cuDNN Version: 8.1.
[2022-12-15 00:58:55,379] [    INFO] - Weights from pretrained model not used in BertModel: ['cls.predictions.decoder_weight', 'cls.predictions.decoder_bias', 'cls.predictions.transform.weight', 'cls.predictions.transform.bias', 'cls.predictions.layer_norm.weight', 'cls.predictions.layer_norm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']


In [6]:
preds = model(batch)

print(f"type of preds: {type(preds)}")
print(f"len of preds: {len(preds)}")
print(f"type of preds: {type(preds)}")
print(f"keys of preds: {preds.keys()}")

W1215 00:59:05.578862 1370870 gpu_resources.cc:201] WARNING: device:  . The installed Paddle is compiled with CUDNN 8.4, but CUDNN version in your machine is 8.1, which may cause serious incompatible bug. Please recompile or reinstall Paddle with compatible CUDNN version.


type of preds: <class 'dict'>
len of preds: 2
type of preds: <class 'dict'>
keys of preds: dict_keys(['loss', 'pred_relations'])


In [6]:
from ppocr.postprocess import build_post_process

config = {
    "name": "VQAReTokenLayoutLMPostProcess",
}

batch_numpy = [out.numpy() for out in batch]
post_process_class = build_post_process(config, global_config)
post_result = post_process_class(preds, batch_numpy)
print(f"len of post_result: {len(post_result)}")
print(f"type of post_result: {type(post_result)}")

len of post_result: 3
type of post_result: <class 'tuple'>


In [7]:
from ppocr.metrics import build_metric

config = {
    "name": "VQAReTokenMetric",
    "main_indicator": "hmean",
}

eval_class = build_metric(config)
eval_class(post_result, batch)
metric = eval_class.get_metric()
print(metric)

{'precision': 0, 'recall': 0, 'hmean': 0}
